In [9]:
import torch 
import math

def rope(X):
    batch_size, seq_len, D = X.shape # 假设D为偶数
    device = X.device
    # 算频率
    half_D = D // 2
    k = torch.arange(half_D, dtype = torch.float32)
    omega = 10000 ** (-2 * k / D)
    
    # 算旋转角度
    positions = torch.arange(seq_len, dtype = torch.float32).unsqueeze(1)
    theta = positions * omega.unsqueeze(0)
    
    sin_theta = torch.sin(theta)
    cos_theta = torch.cos(theta)
    
    X_even = X[:, :, ::2]
    X_odd = X[:, :, 1::2]
    
    sin_theta = sin_theta.unsqueeze(0)
    cos_theta = cos_theta.unsqueeze(0)
    
    X_rotated_even = X_even * cos_theta - X_odd * sin_theta
    X_rotated_odd = X_even * sin_theta + X_odd * cos_theta
    
    X_rotated = torch.zeros_like(X)
    X_rotated[:, :, ::2] = X_rotated_even
    X_rotated[:, :, 1::2] = X_rotated_odd

    return X_rotated

In [21]:
batch_size = 2
seq_len = 5
D = 8
X = torch.randn(batch_size, seq_len, D)
X_rope = rope(X)

print(X, '\n', X_rope)

tensor([[[-0.8170, -0.4779, -1.0475,  2.2702,  0.6012,  1.1450, -1.2604,
           1.0897],
         [-0.5066, -0.2549,  0.7015,  0.6688,  0.9594,  1.5444,  0.7408,
          -1.2212],
         [ 2.0486, -1.3353, -0.2815,  0.9975,  1.7612,  0.2458, -0.9603,
           0.2181],
         [ 0.5611,  0.8523, -1.0063, -1.1259, -1.1981, -1.4968,  1.2107,
          -0.9057],
         [-1.0016, -0.2457,  1.0376,  0.0800, -0.3436, -0.6141,  1.2578,
           0.3163]],

        [[ 0.7772,  1.2440,  0.7100,  1.6108,  0.2929,  0.6903,  0.2893,
           1.6307],
         [-0.2647,  1.8713,  0.3591, -1.1340,  0.2776,  0.0656,  0.0433,
          -2.3983],
         [-1.0501,  1.2874, -0.4278,  0.0693,  0.8855, -1.0952,  0.6083,
           1.4694],
         [ 0.3900, -1.7760, -1.2392, -2.4539,  0.5714,  0.4796, -0.3619,
          -1.0705],
         [-0.2535, -1.0494, -1.9634, -0.3242,  0.1810, -0.5950, -0.1684,
           0.0545]]]) 
 tensor([[[-0.8170, -0.4779, -1.0475,  2.2702,  0.6012,  1.1450, 